# Data Processing

In this notebook, I will process the data which were downloaded from the two sources:
- Brazilian Institute of Geography and Statistics (IBGE)
- Brazilian National Institute of Space Research (INPE)

In other words, I will clean, add features, transform and merge datasets, in order to come up with final datasets which will be used in the Data Analysis notebook in order to create insights from the data.

## 1. Importing raw data

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Annual deforestation dataset
df = pd.read_csv("data/raw_data/df_annual_deforestation.csv", delimiter=';')
df.head(2)

,year,area km²,uf
0,2020,"25,00",TOCANTINS
1,2020,"297,00",RORAIMA


In [2]:
# Population dataset
dfpopul = pd.read_csv("data/raw_data/ALB_Pop_1988_2020.csv")
pd.set_option("display.max_columns", 100)
dfpopul.head(2)

,place,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Rondônia,NaN,1021229,NaN,NaN,1190739,1241693,1291214,1339507,NaN,1255522,1276173,1296856,1317614,1407886,1431777,1455907,1562085,1534594,1562417,NaN,1493565,1503928,NaN,1576455,1590011,1728214,1748531,1768204,1787279,1805788,1757589,1777225,1796460
1,Acre,NaN,411984,NaN,NaN,428006,437495,446480,455242,NaN,500185,514050,527937,541873,574355,586942,600595,630328,669736,686652,NaN,680075,691132,NaN,746386,758786,776463,790101,803513,816687,829619,869265,881935,894470


In [3]:
# State GDP per capita dataset
df_spibpc = pd.read_csv("data/raw_data/PIB/PIBpcpt_estadual.csv")
df_spibpc.head(2)

,year,uf,PIBpcpt
0,1999,RONDÔNIA,3541.0
1,2000,RONDÔNIA,3888.0


In [4]:
df_spibpc.year.unique()

array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2010, 2011, 2012, 2013])

The dataset above lacks the entries for PIBpcpt for the following years: 2006, 2007, 2008, 2009, 2014, 2015, 2016, 2017 and 2018. In order to fill this gap, we need to complement with the data in the file PIB_missing.csv:

In [5]:
# Missing GDP dataset
df_pibmiss = pd.read_csv("data/raw_data/PIB/PIB_missing.csv")
df_pibmiss.head(2)

,year,uf,PIB
0,2006,RONDÔNIA,13107441
1,2007,RONDÔNIA,15002734


In [6]:
# HDI dataset
dfidhm_1991 = pd.read_csv('data/raw_data/IDHM/1991_IDHM.csv')
dfidhm_2000 = pd.read_csv('data/raw_data/IDHM/2000_IDHM.csv')
dfidhm_2010 = pd.read_csv('data/raw_data/IDHM/2010_IDHM.csv')
dfidhm_2017 = pd.read_csv('data/raw_data/IDHM/2017_IDHM.csv')
dfidhm_1991.head(2)

,Territorialidade,Posição IDHM,IDHM,Posição IDHM Renda,IDHM Renda,Posição IDHM Educação,IDHM Educação,Posição IDHM Longevidade,IDHM Longevidade,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,Distrito Federal,1,0.616,2,0.731,1,0.419,1,0.762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,São Paulo,2,0.578,3,0.730,3,0.363,2,0.729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Cattle population dataset
df_c = pd.read_csv("data/raw_data/cabecas_gado.csv")
df_c.head(2)

,place,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Brasil,92495364.0,102531758.0,107349052.0,107296556.0,106942565.0,109177486.0,118971418.0,121785084.0,123487834.0,124185999.0,127654597.0,128422666.0,132221568.0,135726280.0,139599106.0,144154103,147102314.0,152135505,154229303,155134073.0,158243229,161227938,158288540,161416157,163154357,164621038,169875524,176388726,185348838,195551576,204512737,207156696,205886244,199752014,202306731,205307954,209541109,212815311,211279082,211764292,212366132,215220508,218190768,215003578,213809445,214893800
1,Rondônia,41030.0,51507.0,65997.0,74889.0,116617.0,176221.0,250286.0,254348.0,347279.0,575083.0,693663.0,764299.0,884307.0,1051067.0,1247898.0,1594201,1718697.0,2826403,2773896,3286112.0,3469519,3928027,3937291,4330932,5104233,5441734,5664320,6605034,8039890,9392354,10671440,11349452,11484162,11007613,11176201,11532891,11842073,12182259,12218437,12329971,12744326,13397970,13682200,14091378,14367161,14349219


## 2. Transforming and creating new datasets

### Population dataset
In the population dataset df_popul, notice that the entries for the years 2010, 2007, 1996, 1991, 1990, 1988 are empty. In the following, I use a simple approach to fill in these gaps:

In [8]:
dfpopul["2010"] = ((dfpopul["2009"]+dfpopul["2011"])/2).astype(int)
dfpopul["2007"] = ((dfpopul["2006"]+dfpopul["2008"])/2).astype(int)
dfpopul["1996"] = ((dfpopul["1995"]+dfpopul["1997"])/2).astype(int)
dfpopul["1991"] = ((dfpopul["1989"]*0.3)+(dfpopul["1992"]*0.7)).astype(int)
dfpopul["1990"] = ((dfpopul["1989"]*0.7)+(dfpopul["1992"]*0.3)).astype(int)
dfpopul["1988"] = (dfpopul["1989"] - (dfpopul["1990"]-dfpopul["1989"])).astype(int)
dfpopul.place = dfpopul.place.str.upper()
dfpopul.head(2)

,place,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,RONDÔNIA,970376,1021229,1072082,1139886,1190739,1241693,1291214,1339507,1297514,1255522,1276173,1296856,1317614,1407886,1431777,1455907,1562085,1534594,1562417,1527991,1493565,1503928,1540191,1576455,1590011,1728214,1748531,1768204,1787279,1805788,1757589,1777225,1796460
1,ACRE,407178,411984,416790,423199,428006,437495,446480,455242,477713,500185,514050,527937,541873,574355,586942,600595,630328,669736,686652,683363,680075,691132,718759,746386,758786,776463,790101,803513,816687,829619,869265,881935,894470


Now, I will create a new population dataset which will be more convenient to handle, since it will treat the "years" as one single feature:

In [9]:
# Final dataset of estimated population
years = list(df.year.sort_values().astype(str).unique())
dfpop = pd.DataFrame(columns=["year","uf","pop"])

for state in dfpopul.place.unique():
    for year in years:
        #print(dfpop[dfpop.place==state][year].values[0],state,year)
        dic = {"year":int(year), "uf":state, "pop": dfpopul[dfpopul.place==state][year].values[0]}
        dfpop = dfpop.append(dic,ignore_index=True)
dfpop.head(2)

,year,uf,pop
0,1988,RONDÔNIA,970376
1,1989,RONDÔNIA,1021229


### State GDP per capita dataset
The gaps for the GDP per capita data related to the years 2006, 2007, 2008, 2009, 2014, 2015, 2016, 2017 and 2018 need to be filled in. For that, the population dataset *dfpop* and the missing pib dataset *df_pibmiss* will be used.

First, I will compute the GDP per capita related to those missing years, by diving the missing GDP in *df_pibmiss* by the population in *dfpop* for each respective year. A new feature in the dataset df_pibmiss to store those values will be created.

In [10]:
dfpibpop=pd.merge(df_pibmiss,dfpop)
dfpibpop['PIBpcpt'] = (dfpibpop['PIB']*1000)/dfpibpop['pop']
df_spibpc_miss = dfpibpop.drop(['PIB','pop'], axis=1)

# Merging GDP per capita dataset will the missing GDP per capita dataset:
df_spibpc_final = pd.concat([df_spibpc,df_spibpc_miss], ignore_index=True)
df_spibpc_final = df_spibpc_final.sort_values(by=['uf','year']).reset_index(drop=True)
df_spibpc_final['PIBpcpt'] = df_spibpc_final['PIBpcpt'].astype(int)
df_spibpc_final.head(2)

,year,uf,PIBpcpt
0,1999,ACRE,2865
1,2000,ACRE,3048


Next, the variations in the GDP per capita will be calculated:

In [11]:
df_spibpc_final['PIBpcpt_rate'] = np.NAN
for state in df_spibpc_final.uf.unique():
    for yr in df_spibpc_final.year.unique():
        if yr>1999:
            current_GDP = df_spibpc_final[(df_spibpc_final.year==yr)&(df_spibpc_final.uf==state)].PIBpcpt.values[0]
            previous_GDP = df_spibpc_final[(df_spibpc_final.year==(yr-1))&(df_spibpc_final.uf==state)].PIBpcpt.values[0]
            idx = df_spibpc_final[(df_spibpc_final.year==yr)&(df_spibpc_final.uf==state)].index[0]
            var_GDP = current_GDP - previous_GDP
            df_spibpc_final.loc[idx,'PIBpcpt_rate']=var_GDP
df_spibpc_final.head(2)

,year,uf,PIBpcpt,PIBpcpt_rate
0,1999,ACRE,2865,NaN
1,2000,ACRE,3048,183.0


### Annual Deforestation dataset

There are some adjustments that we need to make in the annual deforestation dataset: we can simplify the name of the feature "area km²", and we should transform the numerical notation that uses periods and commas in a fashion that differs from that adopted in Python (or the American numerical notation).

In [12]:
# Changing name of area column:
df.columns = ['year','area_rate','uf']
# Changing from the Brazilian to the American numerical notation:
df['area_rate'] = df['area_rate'].str.replace('.','',regex=False);
df['area_rate'] = df['area_rate'].str.replace(',','.',regex=False);
# Transforming from string to float:
df['area_rate'] = pd.to_numeric(df['area_rate']).astype(int)
# Merging deforestation, population and GDP datasets:
df_defor_pop = pd.merge(df,dfpop).sort_values(by=['uf','year']).reset_index(drop=True)
df_all = pd.merge(df_defor_pop, df_spibpc_final, how='outer')
df_all.head(2)

,year,area_rate,uf,pop,PIBpcpt,PIBpcpt_rate
0,1988,620,ACRE,407178,NaN,NaN
1,1989,540,ACRE,411984,NaN,NaN


### HDI dataset

Now the HDI dataset need to be cleaned.

In [13]:
# Selecting and adding features:
features = ['Territorialidade', 'Posição IDHM', 'IDHM', 'Posição IDHM Renda', 'IDHM Renda',
            'Posição IDHM Educação', 'IDHM Educação', 'Posição IDHM Longevidade', 'IDHM Longevidade']
dfidhm_1991 = dfidhm_1991[features]; dfidhm_1991['year'] = 1991
dfidhm_2000 = dfidhm_2000[features]; dfidhm_2000['year'] = 2000
dfidhm_2010 = dfidhm_2010[features]; dfidhm_2010['year'] = 2010
dfidhm_2017 = dfidhm_2017[features]; dfidhm_2017['year'] = 2017

d1 = pd.concat([dfidhm_1991,dfidhm_2000], ignore_index=True)
d2 = pd.concat([d1,dfidhm_2010], ignore_index=True)
d3 = pd.concat([d2,dfidhm_2017], ignore_index=True)
d3.head(2)

,Territorialidade,Posição IDHM,IDHM,Posição IDHM Renda,IDHM Renda,Posição IDHM Educação,IDHM Educação,Posição IDHM Longevidade,IDHM Longevidade,year
0,Distrito Federal,1,0.616,2,0.731,1,0.419,1,0.762,1991
1,São Paulo,2,0.578,3,0.730,3,0.363,2,0.729,1991


In [14]:
# Changing name of area column:
d3.columns = ['uf', 'IDHM_rank', 'IDHM', 'IDHM_income_rank', 'IDHM_income',
            'IDHM_edu_rank', 'IDHM_edu', 'IDHM_long_rank', 'IDHM_long','year']
# Putting the year column in the first position:
d3 = d3[['year','uf', 'IDHM_rank', 'IDHM', 'IDHM_income_rank', 'IDHM_income',
            'IDHM_edu_rank', 'IDHM_edu', 'IDHM_long_rank', 'IDHM_long']]
# Changing names of states from lower case to upper case:
d3.uf = d3.uf.str.upper()
d3.head(2)

,year,uf,IDHM_rank,IDHM,IDHM_income_rank,IDHM_income,IDHM_edu_rank,IDHM_edu,IDHM_long_rank,IDHM_long
0,1991,DISTRITO FEDERAL,1,0.616,2,0.731,1,0.419,1,0.762
1,1991,SÃO PAULO,2,0.578,3,0.730,3,0.363,2,0.729


Now let us compute the variations in the HDI:

In [15]:
d3['IDHM_rate']        = np.NAN
d3['IDHM_income_rate'] = np.NAN
d3['IDHM_edu_rate']    = np.NAN
d3['IDHM_long_rate']   = np.NAN
yrs = list(d3.year.unique())
for state in d3.uf.unique():
    for i in range(len(yrs)):
        if i>0:
            idx = d3[(d3.year==yrs[i])&(d3.uf==state)].index[0]
            # IDHM
            current_IDHM = d3[(d3.year==yrs[i])&(d3.uf==state)].IDHM.values[0]
            previous_IDHM = d3[(d3.year==yrs[i-1])&(d3.uf==state)].IDHM.values[0]        
            var_IDHM = current_IDHM - previous_IDHM
            d3.loc[idx,'IDHM_rate']=var_IDHM
            # IDHM income:
            current_IDHM_income = d3[(d3.year==yrs[i])&(d3.uf==state)].IDHM_income.values[0]
            previous_IDHM_income = d3[(d3.year==yrs[i-1])&(d3.uf==state)].IDHM_income.values[0]
            var_IDHM_income = current_IDHM_income - previous_IDHM_income
            d3.loc[idx,'IDHM_income_rate']=var_IDHM_income
            # IDHM education:
            current_IDHM_edu = d3[(d3.year==yrs[i])&(d3.uf==state)].IDHM_edu.values[0]
            previous_IDHM_edu = d3[(d3.year==yrs[i-1])&(d3.uf==state)].IDHM_edu.values[0]
            var_IDHM_edu = current_IDHM_edu - previous_IDHM_edu
            d3.loc[idx,'IDHM_edu_rate']=var_IDHM_edu
            # IDHM life expectancy:
            current_IDHM_long = d3[(d3.year==yrs[i])&(d3.uf==state)].IDHM_long.values[0]
            previous_IDHM_long = d3[(d3.year==yrs[i-1])&(d3.uf==state)].IDHM_long.values[0]
            var_IDHM_long = current_IDHM_long - previous_IDHM_long
            d3.loc[idx,'IDHM_long_rate']=var_IDHM_long
# Checking to see if the features were added as expected:
d3[d3.uf=="ACRE"]

,year,uf,IDHM_rank,IDHM,IDHM_income_rank,IDHM_income,IDHM_edu_rank,IDHM_edu,IDHM_long_rank,IDHM_long,IDHM_rate,IDHM_income_rate,IDHM_edu_rate,IDHM_long_rate
20,1991,ACRE,21,0.402,12,0.645,21,0.176,16,0.574,NaN,NaN,NaN,NaN
47,2000,ACRE,19,0.517,17,0.694,22,0.325,15,0.612,0.115,0.049,0.149,0.038
74,2010,ACRE,21,0.663,20,0.777,21,0.559,20,0.671,0.146,0.083,0.234,0.059
101,2017,ACRE,20,0.719,12,0.821,18,0.682,22,0.664,0.056,0.044,0.123,-0.007


### Cattle population dataset

In [16]:
pd.set_option("display.max_columns", 100)
df_c.place = df_c.place.str.upper()
df_c.head(2)

,place,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,BRASIL,92495364.0,102531758.0,107349052.0,107296556.0,106942565.0,109177486.0,118971418.0,121785084.0,123487834.0,124185999.0,127654597.0,128422666.0,132221568.0,135726280.0,139599106.0,144154103,147102314.0,152135505,154229303,155134073.0,158243229,161227938,158288540,161416157,163154357,164621038,169875524,176388726,185348838,195551576,204512737,207156696,205886244,199752014,202306731,205307954,209541109,212815311,211279082,211764292,212366132,215220508,218190768,215003578,213809445,214893800
1,RONDÔNIA,41030.0,51507.0,65997.0,74889.0,116617.0,176221.0,250286.0,254348.0,347279.0,575083.0,693663.0,764299.0,884307.0,1051067.0,1247898.0,1594201,1718697.0,2826403,2773896,3286112.0,3469519,3928027,3937291,4330932,5104233,5441734,5664320,6605034,8039890,9392354,10671440,11349452,11484162,11007613,11176201,11532891,11842073,12182259,12218437,12329971,12744326,13397970,13682200,14091378,14367161,14349219


Missing data for Mato Grosso do Sul for 1974-1977. Tocantins is also missing data for 1974-1988, as well as Roraima for 1990 and 1993. 

Since the deforestation data starts at 1988, we are going to discard the years before 1988 in the cattle population dataset:

In [17]:
df_c = df_c.drop(['1974','1975','1976','1977','1978','1979','1980','1981','1982','1983','1984','1985','1986','1987'], axis=1)

In [18]:
# Modified dataset of cattle population
years = ['1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019']
ufs = ['ACRE','AMAPÁ','AMAZONAS','MARANHÃO','MATO GROSSO','PARÁ','RONDÔNIA','RORAIMA','TOCANTINS'] 

df_cattle = pd.DataFrame(columns=['year','uf','cattle_pop'])
for uf in ufs:
    for year in years:
        dic = {'year':int(year), 'uf':uf, 'cattle_pop': df_c[df_c.place==uf][year].values[0]}
        df_cattle = df_cattle.append(dic,ignore_index=True)
df_cattle.head(2)

,year,uf,cattle_pop
0,1988,ACRE,386995.0
1,1989,ACRE,393925.0


To handle the missing data for Tocantins and Roraima, we will do the following:
- The missing entry for Tocantins in 1988 will be replaced by Tocantins' entry for 1989 subtracted by the difference between the Tocantins' entries for 1990 and 1989;
- The missing entry for Roraima in 1990 will be replaced by the average between Roraima's entries for 1989 and 1991, while Roraima's 1993 will be replaced by the average between Roraima's entries 1992 and 1994.

In [19]:
df_cattle[(df_cattle.year==1988)&(df_cattle.uf=='TOCANTINS')].index[0]

256

In [20]:
# Filling in missing data for Tocantins:
idx = df_cattle[(df_cattle.year==1988)&(df_cattle.uf=='TOCANTINS')].index[0]
toc90 = df_cattle[(df_cattle.year==1990)&(df_cattle.uf=='TOCANTINS')]['cattle_pop'].values[0]
toc89 = df_cattle[(df_cattle.year==1989)&(df_cattle.uf=='TOCANTINS')]['cattle_pop'].values[0]
df_cattle.loc[idx,'cattle_pop'] = toc89-(toc90-toc89)
# Filling in missing data for Roraima:
idx = df_cattle[(df_cattle.year==1990)&(df_cattle.uf=='RORAIMA')]['cattle_pop'].index[0]
ror89 = df_cattle[(df_cattle.year==1989)&(df_cattle.uf=='RORAIMA')]['cattle_pop'].values[0]
ror91 = df_cattle[(df_cattle.year==1991)&(df_cattle.uf=='RORAIMA')]['cattle_pop'].values[0]
df_cattle.loc[idx,'cattle_pop'] = round((ror89 + ror91)/2)
idx = df_cattle[(df_cattle.year==1993)&(df_cattle.uf=='RORAIMA')]['cattle_pop'].index[0]
ror92 = df_cattle[(df_cattle.year==1992)&(df_cattle.uf=='RORAIMA')]['cattle_pop'].values[0]
ror94 = df_cattle[(df_cattle.year==1994)&(df_cattle.uf=='RORAIMA')]['cattle_pop'].values[0]
df_cattle.loc[idx,'cattle_pop'] = round((ror92 + ror94)/2)

In [21]:
df_cattle['cattle_pop'] = df_cattle['cattle_pop'].astype(int)
df_cattle.head(2)

,year,uf,cattle_pop
0,1988,ACRE,386995
1,1989,ACRE,393925


Now we will add a feature related the annual variation in the cattle population:

In [22]:
df_cattle['cattle_rate'] = np.NAN
for state in ufs:
    for yr in df_cattle.year.unique():
        if yr>1988:
            current_pop = df_cattle[(df_cattle.year==yr)&(df_cattle.uf==state)].cattle_pop.values[0]
            previous_pop = df_cattle[(df_cattle.year==(yr-1))&(df_cattle.uf==state)].cattle_pop.values[0]
            idx = df_cattle[(df_cattle.year==yr)&(df_cattle.uf==state)].index[0]
            var_pop = current_pop - previous_pop
            df_cattle.loc[idx,'cattle_rate'] = var_pop
df_cattle.head(2)

,year,uf,cattle_pop,cattle_rate
0,1988,ACRE,386995,NaN
1,1989,ACRE,393925,6930.0


In [23]:
df_final = pd.merge(df_all, df_cattle, how='outer')
df_final.head()

,year,area_rate,uf,pop,PIBpcpt,PIBpcpt_rate,cattle_pop,cattle_rate
0,1988,620,ACRE,407178,NaN,NaN,386995.0,NaN
1,1989,540,ACRE,411984,NaN,NaN,393925.0,6930.0
2,1990,550,ACRE,416790,NaN,NaN,400085.0,6160.0
3,1991,380,ACRE,423199,NaN,NaN,404434.0,4349.0
4,1992,400,ACRE,428006,NaN,NaN,409172.0,4738.0


## 3. Saving datasets

In [24]:
# Dataset containing yearly data regarding deforestation rates, population, GDP per capita, cattle population:
df_final.to_csv('data/data.csv')
# Dataset containing information regarding the HDI for the years of 1999, 2000, 2010 and 2017:
d3.to_csv('data/data_IDHM.csv')